In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR

from tqdm import tqdm
import time
# import cPickle as pickle
from scipy import sparse
from scipy.sparse import hstack

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Функция для вычисления квадратного корня среднеквадратической ошибки логарифма 
# (Root Mean Squared Logarithmic Error (RMSLE))
def rmsle(y, y_pred):
    y_pred[y_pred < 0.0] = 0.0
    log_sqr = np.square(np.log(np.add(y_pred, 1.0)) - np.log(np.add(y, 1.0)))
    return math.sqrt(np.sum(log_sqr) / y.shape[0])

In [3]:
df = pd.read_csv('data/vk_users_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35517 entries, 0 to 35516
Data columns (total 18 columns):
political     5400 non-null float64
country       34114 non-null object
smoking       6264 non-null float64
sex           35517 non-null int64
id            35517 non-null int64
last_name     35517 non-null object
alcohol       6189 non-null float64
religion      6357 non-null object
langs         4624 non-null object
city          32524 non-null object
relation      14826 non-null float64
age           35517 non-null float64
verified      35517 non-null int64
bdate         29173 non-null object
first_name    35517 non-null object
university    14826 non-null float64
life_main     6239 non-null float64
posts         35517 non-null object
dtypes: float64(7), int64(3), object(8)
memory usage: 4.9+ MB


In [4]:
# заменим не указанные значения категориальных признаков на -1
df = df.fillna(-1)
# В столбце university содержится id университета по БД VK. 
# Будем считать, что если он указан, то у пользователя есть высшее 
# образование, иначе нет (хотя строго это не так, он может быть просто 
# не указан. Но примем такое предположение)
df['high_education'] = df['university'].apply(lambda x: 0 if x < -0.5 else 1)
df['age'] = df['age'].astype(int)
df['political'] = df['political'].astype(int)
df['smoking'] = df['smoking'].astype(int)
df['alcohol'] = df['alcohol'].astype(int)
df['relation'] = df['relation'].astype(int)
df['life_main'] = df['life_main'].astype(int)
# df['posts'] = df['posts'].str.decode('utf-8')

In [5]:
# Чтобы превратить нашу задачу в задачу классификации, уменьшим количество категорий возраста
# рассмотрим возраста от 0 до 18; от 18 до 30; от 30 до 50; от 50 до 70 и от 70 до 110 - 5 категорий
def age_cat(age):
    if 0 <= age <= 18:
        return 0
    elif 18 < age <= 30:
        return 1
    elif 30 < age <= 50:
        return 2
    elif 50 < age <= 70:
        return 3
    elif 70 < age <= 110:
        return 4
    
df['age_category'] = df['age'].apply(lambda x: age_cat(x))
df.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education,age_category
0,-1,Россия,-1,1,2615791,Третьякова,-1,-1,-1,Москва,-1,26,0,5.3.1992,Анастасия,-1.00,-1,🌸🌸🌸 #id2615791 (fashionlioness) #модель #фотос...,0,1
1,-1,Россия,-1,2,148071868,Дмитриев,-1,-1,-1,Кострома,-1,18,0,-1,Сергей,-1.00,-1,Карантин)!!!!!!!!!,0,0
2,3,Россия,4,1,54774632,Власова,4,Православие,-1,Пермь,4,110,0,6.7,Анюта,0.00,1,"Не важно, сколько дверей закроется перед твоим...",1,4
3,-1,Россия,-1,1,76303980,Шабалкова,-1,-1,-1,Санкт-Петербург,-1,90,0,3.9,Анастасия,-1.00,-1,Друзья! Я собираю большую посылку с помощью д...,0,4
4,-1,Россия,1,2,104199626,Блейх,1,Православие,"['Русский', 'English']",Санкт-Петербург,1,26,0,6.11,Эдгар,1.00,1,Не мой среди твоих\nИстинный ариец. Характер —...,1,1


In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
df_train.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education,age_category
15714,-1,Украина,-1,1,128537508,Лехновська,-1,-1,-1,Львов,-1,30,0,17.9.1989,Мар'яна,-1.00,-1,"Моя квіточка😘🌹🌼🌸\nКвіточка наша, вітаємо тебе ...",0,1
11666,6,Италия,-1,1,132398387,Петросян,-1,Православие,-1,Fano,1,54,0,7.11,Марина,16542.00,-1,з днем рождения!!!!!\n 😃 Картинки и открытки ♥...,1,3
17047,-1,Беларусь,-1,1,156080718,Соколовская,-1,-1,-1,Минск,-1,50,0,5.7,Елена,-1.00,-1,"Всех с Новым годом Собаки!!! Счастья, Любви,...",0,2
28715,-1,Россия,-1,1,3687543,Игонина,-1,верю в себя,-1,Ульяновск,7,42,0,13.3,Анна,871.00,6,Актуальное время на Новогодние фотосессии и фо...,1,2
27703,-1,Россия,-1,1,819833,Зайцева,-1,верю,-1,Тюмень,4,34,0,27.11,Ксения,862.00,-1,На улице божественная красота😍😍 как у вас посл...,1,2


In [8]:
df_test.head()

,political,country,smoking,sex,id,last_name,alcohol,religion,langs,city,relation,age,verified,bdate,first_name,university,life_main,posts,high_education,age_category
7339,-1,Россия,-1,2,30163255,Кузинский,-1,-1,-1,Сочи,-1,34,0,15.8.1984,Виталик,-1.00,-1,Закрыл соревновательный сезон 17го года. \nСта...,0,2
9058,-1,Россия,-1,2,35057592,Пьеха,-1,-1,-1,Санкт-Петербург,-1,34,1,-1,Стас,-1.00,-1,Я долго ждал- Когда придет её черед? ведь это ...,0,2
29722,-1,Россия,-1,1,116910037,Алдошина,-1,-1,-1,Санкт-Петербург,-1,54,0,20.2.1962,Елена,-1.00,-1,Елена прогнала из рощицы любителей ягод - медв...,0,3
33234,-1,Россия,-1,2,35303,Варламов,-1,-1,-1,Санкт-Петербург,-1,86,0,13.11.1930,Лёша,-1.00,-1,"С днём рождения!!)\nФорд, говоришь? А привод к...",0,4
15361,-1,Россия,-1,2,3159625,Стив,-1,-1,-1,Санкт-Петербург,-1,6,0,666.5,Наволочка,-1.00,-1,СКАЧИВАЕМ И СЛУШАЕМ НАШ MIXTAPE ЗОНА 812!\n\nС...,0,0


In [9]:
# Сохраним наборы данных для дальнейшего использования
df_train.to_csv('learning/df_train.csv', sep='\t', encoding='utf-8')
df_test.to_csv('learning/df_test.csv', sep='\t', encoding='utf-8')

Смысл значений в столбцах sex, political, smoking, alcohol, relation, life_main следующий (из описания API VK):

1) sex - пол пользователя:
1 - женский;
2 - мужской;
0 - пол не указан

2) political - политические предпочтения:

1 - коммунистические;
2 - социалистические;
3 - умеренные;
4 - либеральные;
5 - консервативные;
6 - монархические;
7 - ультраконсервативные;
8 - индиффирентные;
9 - либертарианские;

3) smoking, alcohol - отношение к курению, алкоголю:

1 - резко негативное;
2 - негативное;
3 - компромиссное;
4 - нейтральное;
5 - положительное;

4) relation - семейное положение:

1 - не женат/не замужем;
2 - есть друг/есть подруга;
3 - помолвлен/помолвлена;
4 - женат/замужем;
5 - всё сложно;
6 - в активном поиске;
7 - влюблён/влюблена;
8 - в гражданском браке;
0 - не указано;

5) life_main - главное в жизни:

1 - семья и дети;
2 - карьера и деньги;
3 - развлечения и отдых;
4 - наука и исследования;
5 - совершенствование мира;
6 - саморазвитие;
7 - красота и искусство;
8 - слава и влияние

Попытаемся предсказать возраст (по данным VK (скорее всего, может быть с ошибками: неизвестно, как он там определяется)) по постам пользователя, его полу (пол указан всегда) и указанным признаками из раздела personal, а также по наличию высшего образования

#### Функции преобразования постов пользователя

In [9]:
from nltk.corpus import stopwords

stopwords_en = stopwords.words('english')
stopwords_ru = stopwords.words('russian')
stopwords_ge = stopwords.words('german')

stopwords_all = stopwords_en + stopwords_ru + stopwords_ge

# дополнительные буквосочетания, которые будем удалять из текстов
additional_stopwords = \
[u'https', u'vk', u'com', u'id', u'ph', u'др', u'св', u'ff', u'la', u'это', \
 u'de', u'pa', u'bb', u'p', u'ул', u'ин', u'http', u'ru', u'md', u'x', \
 u'ft', u'сб', u'b', u'к', u'www', u'youtube', u'ка', u'v', u'g', u'goo', u'gl', \
 u'eu', u'u', u'te', u'un', u'вк', u'w', u'ly', u'su', u'bu', u'vl', u'эт', u'r', u'e', \
 u'свой', u'ещё', u'мой', u'весь', u'днём', u'youtu', u'твой', u'наш', u'ваш', u'тот', u'этот']

stopwords_all = stopwords_all + additional_stopwords

# дополнительные удаляемые после лемматизации слова
delete_words = [u'свой', u'ещё', u'мой', u'весь', u'днём', u'youtu', u'твой', u'наш', u'ваш', u'тот', u'этот']

In [10]:
# используем для лемматизации библиотеку pymorphy2, которая работает с русским языком
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    return morph.parse(text)[0].normal_form

def my_tokenizer(text):
    text = text.lower()
    # очистка от html-разметки
    text = re.sub('<[^>]*>', '', text)
    # выделение смайлов
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # удаление несловарных символов
    text = re.sub(r'[\W]+', ' ', text, flags=re.U) + ' '.join(emoticons).replace('-', '')
    # удалим также все цифры, которые вряд ли полезны при анализе текста
    text = re.sub(r'\d+', '', text, flags=re.U)
    # удалим два или более пробелов
    text = re.sub(r'[ ]{2,}', '', text, flags=re.U)
    # удалим подчёркивания
    text = re.sub(r'[_]+', '', text, flags=re.U)
    # разбиение по пробелам и удаление стоп-слов
    tokenized = [w for w in text.split() if w not in stopwords_all]
    tokenized = [re.sub(r"\W", "", w, flags=re.U) for w in tokenized]
    # лемматизация
    tokenized = [lemmatization(w) for w in tokenized]
    # удалим все слова из одной и двух букв. Вряд ли они несут большой смысл. 
    # Также удалим "слова-паразиты"
    tokenized = [w for w in tokenized if len(w) > 2 and w not in delete_words]
    return tokenized

In [11]:
def get_posts_col(df):
    return df[['posts']]

def get_sex_col(df):
    return df[['sex']]

def get_categorial_cols(df):
    return df[['high_education', 'political', 'smoking', 'alcohol', 'relation', 'life_main']]

vec = make_union(*[
    make_pipeline(FunctionTransformer(get_categorial_cols, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_posts_col, validate=False)), CountVectorizer(tokenizer=my_tokenizer)
])

С pipeline'ом что-то не получается, поэтому используем только столбец текста

In [12]:
y_train = df_train['age_category']
y_test = df_test['age_category']
print(y_train.shape)
print(y_test.shape)

(28413,)
(7104,)


In [15]:
y_train.to_csv('learning/y_train', sep='\t')
y_test.to_csv('learning/y_test', sep='\t')

In [13]:
# функции для сохранения и последующего восстановления разреженной матрицы
def save_sparse_csr(filename, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [39]:
# vectorizer = CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 2))
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=5000)

In [40]:
X_train_bag_of_words = vectorizer.fit_transform(tqdm(df_train['posts']))
X_train_bag_of_words.shape

100%|██████████| 28413/28413 [38:22<00:00, 12.34it/s]


(28413, 4694)

### Исследуем получившуюся матрицу tf-idf

In [16]:
features = vectorizer.get_feature_names()

In [17]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [18]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [19]:
top_feats_in_doc(X_train_bag_of_words, features, 1000, top_n=6)

,feature,tfidf
0,love,0.61
1,happy,0.36
2,клуб,0.19
3,девочка,0.14
4,муж,0.14
5,деньга,0.13


In [20]:
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [21]:
top_mean_feats(X_train_bag_of_words, features, top_n=10)

,feature,tfidf
0,app,0.04
1,человек,0.04
2,рождение,0.03
3,который,0.03
4,открытка,0.03
5,друг,0.03
6,год,0.03
7,жизнь,0.03
8,любить,0.03
9,новый,0.03


In [22]:
def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [24]:
top_feats_by_class(X_train_bag_of_words, y_train, features, top_n=5)

[    feature  tfidf
 0       app   0.04
 1    узнать   0.03
 2  рождение   0.03
 3      друг   0.03
 4    любить   0.03,      feature  tfidf
 0    человек   0.04
 1  instagram   0.04
 2    который   0.04
 3    спасибо   0.03
 4       друг   0.03,     feature  tfidf
 0       app   0.06
 1  открытка   0.05
 2   человек   0.04
 3   который   0.04
 4       год   0.03,     feature  tfidf
 0       app   0.07
 1  открытка   0.06
 2   человек   0.04
 3   который   0.03
 4  рождение   0.03,     feature  tfidf
 0   человек   0.04
 1  рождение   0.04
 2    любить   0.04
 3   который   0.03
 4     жизнь   0.03]

In [ ]:
save_sparse_csr('learning/X_train_bag_of_words', X_train_bag_of_words)

In [21]:
# сохраним векторизатор, чтобы не тратить время на повторное обучение
with open('learning/vectorizer.pk', 'wb') as fin:
    pickle.dump(vectorizer, fin)

In [13]:
# vectorizer = pickle.load(open('vectorizer.pk', 'rb'))
# vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<function my_tokenizer at 0x7fdf1aef2e60>, use_idf=True,
        vocabulary=None)

In [25]:
# Функция для получения вектора признаков для обучения в виде разреженной матрицы
def get_X(X_bag_of_words, df):
    scaler = MinMaxScaler()

    # прибавим остальные признаки, которые будем учитывать при обучении модели
    X_sex = sparse.csr_matrix(get_sex_col(df))

    categorial_cols_scale = scaler.fit_transform(get_categorial_cols(df))
    X_categorial = sparse.csr_matrix(categorial_cols_scale)

    X = hstack([X_sex, X_categorial, X_bag_of_words])

    print(X.shape)
    
    return X

In [41]:
X_train = get_X(X_train_bag_of_words, df_train)

(28413, 4701)


In [42]:
X_test_bag_of_words = vectorizer.transform(tqdm(df_test['posts']))
save_sparse_csr('learning/X_test_bag_of_words', X_test_bag_of_words)

100%|██████████| 7104/7104 [10:06<00:00, 11.72it/s]


In [43]:
X_test = get_X(X_test_bag_of_words, df_test)

(7104, 4701)


### Обучение модели

In [44]:
def randomized_cv(model, param_grid, x_train, y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy', n_iter=10)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    return grid_search.best_estimator_

In [45]:
param_grid = {'alpha':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 5]}
model = MultinomialNB()
best_model = randomized_cv(model, param_grid, X_train, y_train)

model MultinomialNB best accuracy score is 0.4875233167916095
time for training is 6.673130989074707 seconds


In [31]:
with open('learning/model_bayes.pk', 'wb') as fin:
    pickle.dump(best_model, fin)

NameError: name 'pickle' is not defined

In [46]:
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.485923423423


Видим, что точность не слишком высокая, но всё же лучше случайного гадания (у нас 5 классов, поэтому при случайном гадании было бы 0.2). Есть ощущение, что текств постах ВК не слишком коррелирует с возрастом пользователя, поэтому хорошее предсказание сделать тяжело

In [33]:
# попробуем биграммы
vectorizer_2gram = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(2, 2), analyzer='word', max_features=5000)
X_train_bag_of_words_2gram = vectorizer_2gram.fit_transform(tqdm(df_train['posts']))

100%|██████████| 28413/28413 [39:27<00:00, 12.00it/s]


In [34]:
X_train_2gram = get_X(X_train_bag_of_words_2gram, df_train)

(28413, 31)


In [35]:
X_test_bag_of_words_2gram = vectorizer_2gram.transform(tqdm(df_test['posts']))

100%|██████████| 7104/7104 [09:41<00:00, 12.22it/s]


In [36]:
X_test_2gram = get_X(X_test_bag_of_words_2gram, df_test)

(7104, 31)


In [37]:
model_2gram = MultinomialNB()
best_model_2gram = randomized_cv(model_2gram, param_grid, X_train_2gram, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.5/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


model MultinomialNB best accuracy score is 0.4162883187273431
time for training is 0.4641273021697998 seconds


In [38]:
y_pred_2gram = best_model_2gram.predict(X_test_2gram)
print(accuracy_score(y_test, y_pred_2gram))

0.420467342342
